System information (for reproducibility):

In [119]:
versioninfo()

Julia Version 1.11.5
Commit 760b2e5b739 (2025-04-14 06:53 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin24.0.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, apple-m2)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [120]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2025spring/slides/18-cg`


Status `~/Documents/github.com/ucla-biostat-257/2025spring/slides/18-cg/Project.toml`
⌅ [2169fc97] AlgebraicMultigrid v0.5.1
  [7d9fca2a] Arpack v0.5.4
  [6e4b80f9] BenchmarkTools v1.6.0
  [42fd0dbc] IterativeSolvers v0.9.4
  [ba0b0d4f] Krylov v0.10.0
  [7a12625a] LinearMaps v3.11.4
  [b51810bb] MatrixDepot v1.0.13
  [af69fa37] Preconditioners v0.6.1
  [b8865327] UnicodePlots v3.7.2
  [37e2e46d] LinearAlgebra v1.11.0
  [9a3f8284] Random v1.11.0
  [2f01184e] SparseArrays v1.11.0
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


In [121]:
using AlgebraicMultigrid, BenchmarkTools, IterativeSolvers, 
    Krylov, LinearAlgebra, MatrixDepot, Random, SparseArrays

## Introduction

* Conjugate gradient is the top-notch iterative method for solving large, **structured** linear systems $\mathbf{A} \mathbf{x} = \mathbf{b}$, where $\mathbf{A}$ is pd.  
Earlier we talked about Jacobi, Gauss-Seidel, and successive over-relaxation (SOR) as the classical iterative solvers. They are rarely used in practice due to slow convergence.  

    [Kershaw's results](http://www.sciencedirect.com/science/article/pii/0021999178900980?via%3Dihub) for a fusion problem.

| Method                                 | Number of Iterations |
|----------------------------------------|----------------------|
| Gauss Seidel                           | 208,000              |
| Block SOR methods                      | 765                  |
| Incomplete Cholesky **conjugate gradient** | 25                   |


* History: Hestenes (**UCLA** professor!) and Stiefel proposed conjugate gradient method in 1950s.

Hestenes and Stiefel (1952), [Methods of conjugate gradients for solving linear systems](http://nvlpubs.nist.gov/nistpubs/jres/049/jresv49n6p409_A1b.pdf), _Jounral of Research of the National Bureau of Standards_.

* Solve linear equation $\mathbf{A} \mathbf{x} = \mathbf{b}$, where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is **pd**, is equivalent to 
$$
\begin{eqnarray*}
	\text{minimize} \,\, f(\mathbf{x}) = \frac 12 \mathbf{x}^T \mathbf{A} \mathbf{x} - \mathbf{b}^T \mathbf{x}.
\end{eqnarray*}
$$
Denote $\nabla f(\mathbf{x}) = \mathbf{A} \mathbf{x} - \mathbf{b} =: r(\mathbf{x})$.

## Conjugate gradient (CG) method

* Consider a simple idea: coordinate descent, that is to update components $x_j$ alternatingly. Same as the Gauss-Seidel iteration. Usually it takes too many iterations.

<img src="coordinate_descent.png" width="400" align="center"/>

* A set of vectors $\{\mathbf{p}^{(0)},\ldots,\mathbf{p}^{(t)}\}$ are said to be **conjugate with respect to $\mathbf{A}$** if
$$
\begin{eqnarray*}
	\mathbf{p}_i^T \mathbf{A} \mathbf{p}_j = 0, \quad \text{for all } i \ne j.
\end{eqnarray*}
$$
For example, eigen-vectors of $\mathbf{A}$ are conjugate to each other. Why?

* **Conjugate direction** method: Given a set of conjugate vectors $\{\mathbf{p}^{(0)},\ldots,\mathbf{p}^{(n-1)}\}$, at iteration $t$, we search along the conjugate direction $\mathbf{p}^{(t)}$
$$
\begin{eqnarray*}
	\mathbf{x}^{(t+1)} = \mathbf{x}^{(t)} + \alpha^{(t)} \mathbf{p}^{(t)},
\end{eqnarray*}
$$
where
$$
\begin{eqnarray*}
	\alpha^{(t)} = - \frac{\mathbf{r}^{(t)T} \mathbf{p}^{(t)}}{\mathbf{p}^{(t)T} \mathbf{A} \mathbf{p}^{(t)}}
\end{eqnarray*}
$$
is the optimal step length.

* Theorem: In conjugate direction method, $\mathbf{x}^{(t)}$ converges to the solution in **at most** $n$ steps.

    Intuition: Look at graph.
    
<img src="conjugate_direction.png" width="400" align="center"/>

* **Conjugate gradient** method. Idea: generate $\mathbf{p}^{(t)}$ using only $\mathbf{p}^{(t-1)}$
$$
\begin{eqnarray*}
	\mathbf{p}^{(t)} = - \mathbf{r}^{(t)} + \beta^{(t)} \mathbf{p}^{(t-1)},
\end{eqnarray*}
$$
where $\beta^{(t)}$ is determined by the conjugacy condition $\mathbf{p}^{(t-1)T} \mathbf{A} \mathbf{p}^{(t)} = 0$
$$
\begin{eqnarray*}
	\beta^{(t)} = \frac{\mathbf{r}^{(t)T} \mathbf{A} \mathbf{p}^{(t-1)}}{\mathbf{p}^{(t-1)T} \mathbf{A} \mathbf{p}^{(t-1)}}.
\end{eqnarray*}
$$

* **CG algorithm (preliminary version)**:  

    0. Given $\mathbf{x}^{(0)}$
    0. Initialize: $\mathbf{r}^{(0)} \gets \mathbf{A} \mathbf{x}^{(0)} - \mathbf{b}$, $\mathbf{p}^{(0)} \gets - \mathbf{r}^{(0)}$, $t=0$
    0. While $\mathbf{r}^{(t)} \ne \mathbf{0}$
        1. $\alpha^{(t)} \gets - \frac{\mathbf{r}^{(t)T} \mathbf{p}^{(t)}}{\mathbf{p}^{(t)T} \mathbf{A} \mathbf{p}^{(t)}}$
        2. $\mathbf{x}^{(t+1)} \gets \mathbf{x}^{(t)} + \alpha^{(t)} \mathbf{p}^{(t)}$
        3. $\mathbf{r}^{(t+1)} \gets \mathbf{A} \mathbf{x}^{(t+1)} - \mathbf{b}$
        4. $\beta^{(t+1)} \gets \frac{\mathbf{r}^{(t+1)T} \mathbf{A} \mathbf{p}^{(t)}}{\mathbf{p}^{(t)T} \mathbf{A} \mathbf{p}^{(t)}}$
        5. $\mathbf{p}^{(t+1)} \gets - \mathbf{r}^{(t+1)} + \beta^{(t+1)} \mathbf{p}^{(t)}$
        6. $t \gets t+1$
        
    Remark: The initial conjugate direction $\mathbf{p}^{(0)} \gets - \mathbf{r}^{(0)}$ is crucial.
        
* Theorem: With CG algorithm
    0. $\mathbf{r}^{(t)}$ are mutually orthogonal. 
    0. $\{\mathbf{r}^{(0)},\ldots,\mathbf{r}^{(t)}\}$ is contained in the **Krylov subspace** of degree $t$ for $\mathbf{r}^{(0)}$, denoted by
    $$
    \begin{eqnarray*}
        {\cal K}(\mathbf{r}^{(0)}; t) = \text{span} \{\mathbf{r}^{(0)},\mathbf{A} \mathbf{r}^{(0)}, \mathbf{A}^2 \mathbf{r}^{(0)}, \ldots, \mathbf{A}^{t} \mathbf{r}^{(0)}\}.
    \end{eqnarray*}
    $$
    0. $\{\mathbf{p}^{(0)},\ldots,\mathbf{p}^{(t)}\}$ is contained in ${\cal K}(\mathbf{r}^{(0)}; t)$. 
    0. $\mathbf{p}^{(0)}, \ldots, \mathbf{p}^{(t)}$ are conjugate with respect to $\mathbf{A}$.  
The iterates $\mathbf{x}^{(t)}$ converge to the solution in at most $n$ steps.

* **CG algorithm (economical version)**: saves one matrix-vector multiplication.

    0. Given $\mathbf{x}^{(0)}$
    0. Initialize: $\mathbf{r}^{(0)} \gets \mathbf{A} \mathbf{x}^{(0)} - \mathbf{b}$, $\mathbf{p}^{(0)} \gets - \mathbf{r}^{(0)}$, $t=0$
    0. While $\mathbf{r}^{(t)} \ne \mathbf{0}$
        1. $\alpha^{(t)} \gets \frac{\mathbf{r}^{(t)T} \mathbf{r}^{(t)}}{\mathbf{p}^{(t)T} \mathbf{A} \mathbf{p}^{(t)}}$
        2. $\mathbf{x}^{(t+1)} \gets \mathbf{x}^{(t)} + \alpha^{(t)} \mathbf{p}^{(t)}$
        3. $\mathbf{r}^{(t+1)} \gets \mathbf{r}^{(t)} + \alpha^{(t)} \mathbf{A} \mathbf{p}^{(t)}$
        4. $\beta^{(t+1)} \gets \frac{\mathbf{r}^{(t+1)T} \mathbf{r}^{(t+1)}}{\mathbf{r}^{(t)T} \mathbf{r}^{(t)}}$
        5. $\mathbf{p}^{(t+1)} \gets - \mathbf{r}^{(t+1)} + \beta^{(t+1)} \mathbf{p}^{(t)}$
        6. $t \gets t+1$

* Computation cost per iteration is **one** matrix vector multiplication: $\mathbf{A} \mathbf{p}^{(t)}$.  
Consider PageRank problem, $\mathbf{A}$ has dimension $n \approx 10^{10}$ but is highly structured (sparse + low rank). Each matrix vector multiplication takes $O(n)$.
    
* Theorem: If $\mathbf{A}$ has $r$ distinct eigenvalues, $\mathbf{x}^{(t)}$ converges to solution $\mathbf{x}^*$ in at most $r$ steps.

## Pre-conditioned conjugate gradient (PCG)

* Summary of conjugate gradient method for solving $\mathbf{A} \mathbf{x} = \mathbf{b}$ or equivalently minimizing $\frac 12 \mathbf{x}^T \mathbf{A} \mathbf{x} -  \mathbf{b}^T \mathbf{x}$:
    * Each iteration needs one matrix vector multiplication: $\mathbf{A} \mathbf{p}^{(t+1)}$. For structured $\mathbf{A}$, often $O(n)$ cost per iteration.
    * Guaranteed to converge in $n$ steps.
    
* Two important bounds for conjugate gradient algorithm:

    Let $\lambda_1 \le \cdots \le \lambda_n$ be the ordered eigenvalues of a pd $\mathbf{A}$.  
$$
\begin{eqnarray*}
    \|\mathbf{x}^{(t+1)} - \mathbf{x}^*\|_{\mathbf{A}}^2 &\le& \left( \frac{\lambda_{n-t} - \lambda_1}{\lambda_{n-t} + \lambda_1} \right)^2 \|\mathbf{x}^{(0)} - \mathbf{x}^*\|_{\mathbf{A}}^2 \\
    \|\mathbf{x}^{(t+1)} - \mathbf{x}^*\|_{\mathbf{A}}^2 &\le& 2 \left( \frac{\sqrt{\kappa(\mathbf{A})}-1}{\sqrt{\kappa(\mathbf{A})}+1} \right)^{t} \|\mathbf{x}^{(0)} - \mathbf{x}^*\|_{\mathbf{A}}^2,
\end{eqnarray*}
$$
where $\kappa(\mathbf{A}) = \lambda_n/\lambda_1$ is the condition number of $\mathbf{A}$.

<img src="cg_twocluster_spectrum.png" width="300" align="center"/>

<img src="cg_twocluster_iterates.png" width="300" align="center"/>

* Messages:
    * Roughly speaking, if the eigenvalues of $\mathbf{A}$ occur in $r$ distinct clusters, the CG iterates will _approximately_ solve the problem after $O(r)$ steps.  
    * $\mathbf{A}$ with a small condition number ($\lambda_1 \approx \lambda_n$) converges fast.
    
* **Pre-conditioning**: Change of variables $\widehat{\mathbf{x}} = \mathbf{C} \mathbf{x}$ via a nonsingular $\mathbf{C}$ and solve
$$
	(\mathbf{C}^{-T} \mathbf{A} \mathbf{C}^{-1}) \widehat{\mathbf{x}} = \mathbf{C}^{-T} \mathbf{b}.
$$
Choose $\mathbf{C}$ such that 
    * $\mathbf{C}^{-T} \mathbf{A} \mathbf{C}^{-1}$ has small condition number, or 
    * $\mathbf{C}^{-T} \mathbf{A} \mathbf{C}^{-1}$ has clustered eigenvalues
    * Inexpensive solution of $\mathbf{C}^T \mathbf{C} \mathbf{y} = \mathbf{r}$
    
* Preconditioned CG does not make use of $\mathbf{C}$ explicitly, but rather the matrix $\mathbf{M} = \mathbf{C}^T \mathbf{C}$.

* **Preconditioned CG (PCG)** algorithm: 

    0. Given $\mathbf{x}^{(0)}$, pre-conditioner $\mathbf{M}$
    0. $\mathbf{r}^{(0)} \gets \mathbf{A} \mathbf{x}^{(0)} - \mathbf{b}$
    0. solve $\mathbf{M} \mathbf{y}^{(0)} = \mathbf{r}^{(0)}$ for $\mathbf{y}^{(0)}$
    0. $\mathbf{p}^{(0)} \gets - \mathbf{r}^{(0)}$, $t=0$
    0. While $\mathbf{r}^{(t)} \ne \mathbf{0}$
        1. $\alpha^{(t)} \gets \frac{\mathbf{r}^{(t)T} \mathbf{y}^{(t)}}{\mathbf{p}^{(t)T} \mathbf{A} \mathbf{p}^{(t)}}$
        2. $\mathbf{x}^{(t+1)} \gets \mathbf{x}^{(t)} + \alpha^{(t)} \mathbf{p}^{(t)}$
        3. $\mathbf{r}^{(t+1)} \gets \mathbf{r}^{(t)} + \alpha^{(t)} \mathbf{A} \mathbf{p}^{(t)}$
        4. Solve $\mathbf{M} \mathbf{y}^{(t+1)} = \mathbf{r}^{(t+1)}$ for $\mathbf{y}^{(t+1)}$
        5. $\beta^{(t+1)} \gets \frac{\mathbf{r}^{(t+1)T} \mathbf{y}^{(t+1)}}{\mathbf{r}^{(t)T} \mathbf{r}^{(t)}}$
        6. $\mathbf{p}^{(t+1)} \gets - \mathbf{y}^{(t+1)} + \beta^{(t+1)} \mathbf{p}^{(t)}$
        7. $t \gets t+1$

    Remark: Only extra cost in the pre-conditioned CG algorithm is the need to solve the linear system $\mathbf{M} \mathbf{y} = \mathbf{r}$.
    
* Pre-conditioning is more like an art than science. Some choices include     
    * Incomplete Cholesky. $\mathbf{A} \approx \tilde{\mathbf{L}} \tilde{\mathbf{L}}^T$, where $\tilde{\mathbf{L}}$ is a sparse approximate Cholesky factor. Then $\tilde{\mathbf{L}}^{-1} \mathbf{A} \tilde{\mathbf{L}}^{-T} \approx \mathbf{I}$ (perfectly conditioned) and $\mathbf{M} \mathbf{y} = \tilde{\mathbf{L}} \tilde {\mathbf{L}}^T \mathbf{y} = \mathbf{r}$ is easy to solve.  
    * Banded pre-conditioners.  
    * Choose $\mathbf{M}$ as a coarsened version of $\mathbf{A}$.
    * Subject knowledge. Knowledge about the structure and origin of a problem is often the key to devising efficient pre-conditioner. For example, see recent work of Stein, Chen, Anitescu (2012) for pre-conditioning large covariance matrices. http://epubs.siam.org/doi/abs/10.1137/110834469

### Example of PCG

[Preconditioners.jl](https://github.com/mohamed82008/Preconditioners.jl) wraps a bunch of preconditioners.

We use the Wathen matrix (sparse and positive definite) as a test matrix.

In [122]:
# Wathen matrix of dimension 30401 x 30401
A = matrixdepot("wathen", 100)

30401×30401 SparseMatrixCSC{Float64, Int64} with 471601 stored entries:
⎡⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⎦

In [123]:
using UnicodePlots
spy(A)

          ┌──────────────────────────────────────────┐    
        1 │⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
          │⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
          │⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀│  

In [124]:
# sparsity level
count(!iszero, A) / length(A)

0.0005102687577359558

In [125]:
# rhs
b = ones(size(A, 1))
# solve Ax=b by CG
xcg = IterativeSolvers.cg(A, b);
@benchmark IterativeSolvers.cg($A, $b)

BenchmarkTools.Trial: 50 samples with 1 evaluation per sample.
 Range (min … max):   95.556 ms … 116.052 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      97.084 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   100.388 ms ±   5.866 ms  ┊ GC (mean ± σ):  0.05% ± 0.37%

  ▆▁█ ▄                               ▁                          
  ███▆█▇▁▁▆▄▄▁▁▁▁▁▁▁▁▁▁▆▁▁▁▁▁▄▁▁▁▁▁▄▄▁█▄▆▄▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▄▁▁▄ ▁
  95.6 ms          Histogram: frequency by time          116 ms <

 Memory estimate: 1.00 MiB, allocs estimate: 20.

#### Diagonal preconditioner

Compute the diagonal preconditioner:

In [126]:
using Preconditioners

# Diagonal preconditioner
@time p = DiagonalPreconditioner(A)
dump(p)

  0.000205 seconds (33 allocations: 1.732 MiB)
DiagonalPreconditioner{Float64, Vector{Float64}}
  D: Array{Float64}((30401,)) [11.458380664154248, 61.11136354215599, 14.346040960270532, 15.400854912620183, 2.973658010165327, 0.4586544749282319, 9.80270951607837, 51.82246294415641, 13.008211499680696, 17.554665054140635  …  20.427629666471365, 11.742969311459348, 42.20153999464516, 9.301013450638496, 7.403865075426824, 7.0591650159837425, 30.245015009819806, 12.640601905949744, 37.171528488578836, 6.969661591608531]


In [127]:
# solver Ax=b by PCG
xpcg = IterativeSolvers.cg(A, b, Pl = p)
# same answer?
norm(xcg - xpcg)

5.713513500214473e-7

In [128]:
# PCG with diagonal preconditioner is >5 fold faster than CG
@benchmark IterativeSolvers.cg($A, $b, Pl = $p)

BenchmarkTools.Trial: 310 samples with 1 evaluation per sample.
 Range (min … max):  14.523 ms … 21.097 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     15.947 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.138 ms ±  1.299 ms  ┊ GC (mean ± σ):  0.29% ± 1.32%

    ▆█▃   ▂               ▂                                    
  ▄████▇█▇█▅▃▇▃▃▃▄▄▄▅▅▆▄█▇█▅▅▃▃▄▅▅▄▃▃▄▄█▇▇▄▃▃▁▃▁▃▁▃▁▃▁▁▁▁▁▃▁▃ ▄
  14.5 ms         Histogram: frequency by time        19.8 ms <

 Memory estimate: 1.00 MiB, allocs estimate: 20.

#### Incomplete Cholesky preconditioner

Compute the incomplete cholesky preconditioner:

In [129]:
@time p = CholeskyPreconditioner(A, 2)
dump(p)

  0.022275 seconds (82 allocations: 22.010 MiB, 6.67% gc time)
CholeskyPreconditioner{LimitedLDLFactorizations.LimitedLDLFactorization{Float64, Int64, Vector{Int64}, Vector{Int64}}}
  ldlt: LimitedLDLFactorizations.LimitedLDLFactorization{Float64, Int64, Vector{Int64}, Vector{Int64}}
    __factorized: Bool true
    n: Int64 30401
    colptr: Array{Int64}((30402,)) [1, 13, 25, 37, 54, 65, 77, 89, 101, 118  …  265024, 265027, 265031, 265034, 265038, 265041, 265043, 265044, 265044, 265044]
    rowind: Array{Int64}((281402,)) [3, 4, 5, 11, 12, 1557, 1558, 1559, 1599, 4639  …  30394, 30395, 30396, 30396, 30397, 30398, 30398, 30399, 30400, 30400]
    Lrowind: SubArray{Int64, 1, Vector{Int64}, Tuple{UnitRange{Int64}}, true}
      parent: Array{Int64}((281402,)) [3, 4, 5, 11, 12, 1557, 1558, 1559, 1599, 4639  …  30394, 30395, 30396, 30396, 30397, 30398, 30398, 30399, 30400, 30400]
      indices: Tuple{UnitRange{Int64}}
        1: UnitRange{Int64}
          start: Int64 1
          stop: Int64 

Pre-conditioned conjugate gradient:

In [130]:
# solver Ax=b by PCG
xpcg = IterativeSolvers.cg(A, b, Pl = p)
# same answer?
norm(xcg - xpcg)

5.67403826686389e-7

In [131]:
# PCG with incomplete Cholesky is >5 fold faster than CG
@benchmark IterativeSolvers.cg($A, $b, Pl = $p)

BenchmarkTools.Trial: 294 samples with 1 evaluation per sample.
 Range (min … max):  15.899 ms … 23.459 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.553 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.007 ms ±  1.256 ms  ┊ GC (mean ± σ):  0.26% ± 1.71%

   ▅▇█▅     ▂ ▂                                                
  ▅████▇▆▅▄▆███▆▅▃▃▁▂▁▂▂▂▂▁▃▃▂▂▂▃▁▁▂▁▃▁▂▃▃▁▃▁▂▁▃▁▁▁▁▁▁▁▁▁▁▁▁▂ ▃
  15.9 ms         Histogram: frequency by time        22.6 ms <

 Memory estimate: 1.00 MiB, allocs estimate: 20.

#### AMG preconditioner

Let's try the AMG preconditioner.

In [132]:
using AlgebraicMultigrid

@time ml = AMGPreconditioner{RugeStuben}(A) # Construct a Ruge-Stuben solver

  0.026734 seconds (932 allocations: 68.417 MiB, 9.92% gc time)


AMGPreconditioner{RugeStuben, AlgebraicMultigrid.MultiLevel{AlgebraicMultigrid.Pinv{Float64}, GaussSeidel{SymmetricSweep}, GaussSeidel{SymmetricSweep}, SparseMatrixCSC{Float64, Int64}, Adjoint{Float64, SparseMatrixCSC{Float64, Int64}}, SparseMatrixCSC{Float64, Int64}, AlgebraicMultigrid.MultiLevelWorkspace{Vector{Float64}, 1}}, AlgebraicMultigrid.V}(Multilevel Solver
-----------------
Operator Complexity: 1.135
Grid Complexity: 1.171
No. of Levels: 8
Coarse Solver: Pinv
Level     Unknowns     NonZeros
-----     --------     --------
    1        30401       471601 [88.14%]
    2         3609        45325 [ 8.47%]
    3         1084        13008 [ 2.43%]
    4          343         3657 [ 0.68%]
    5          116         1056 [ 0.20%]
    6           39          297 [ 0.06%]
    7           16          108 [ 0.02%]
    8            5           23 [ 0.00%]
, AlgebraicMultigrid.V())

In [133]:
# use AMG preconditioner in CG
xamg = IterativeSolvers.cg(A, b, Pl = ml)
# same answer?
norm(xcg - xamg)

5.746586234519594e-7

In [134]:
@benchmark IterativeSolvers.cg($A, $b, Pl = $ml)

BenchmarkTools.Trial: 99 samples with 1 evaluation per sample.
 Range (min … max):  49.062 ms … 66.851 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     50.088 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   50.698 ms ±  2.450 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▆█▄▂▂▂▆                                                     
  ▄███████▅▇▅▃▄▁▃▄▁▁▁▁▁▁▃▁▃▁▁▃▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▃ ▁
  49.1 ms         Histogram: frequency by time        60.2 ms <

 Memory estimate: 1.00 MiB, allocs estimate: 20.

## Other Krylov subspace methods

* We leant about CG/PCG, which is for solving $\mathbf{A} \mathbf{x} = \mathbf{b}$, $\mathbf{A}$ pd.

* **MINRES (minimum residual method)**: symmetric indefinite $\mathbf{A}$.

* **Bi-CG (bi-conjugate gradient)**: unsymmetric $\mathbf{A}$.

* **Bi-CGSTAB (Bi-CG stabilized)**: improved version of Bi-CG.

* **GMRES (generalized minimum residual method)**: current _de facto_ method for unsymmetric $\mathbf{A}$. E.g., PageRank problem.

* **Lanczos method**: top eigen-pairs of a large symmetric matrix.

* **Arnoldi method**: top eigen-pairs of a large unsymmetric matrix.

* **Lanczos bidiagonalization** algorithm: top singular triplets of large matrix.

* **LSQR**: least square problem $\min \|\mathbf{y} - \mathbf{X} \beta\|_2^2$. Algebraically equivalent to applying CG to the normal equation $(\mathbf{X}^T \mathbf{X} + \lambda^2 I) \beta = \mathbf{X}^T \mathbf{y}$.

* **LSMR**: least square problem $\min \|\mathbf{y} - \mathbf{X} \beta\|_2^2$. Algebraically equivalent to applying MINRES to the normal equation $(\mathbf{X}^T \mathbf{X} + \lambda^2 I) \beta = \mathbf{X}^T \mathbf{y}$.

## Software

### Matlab 

* Iterative methods for solving linear equations:  
    `pcg`, `bicg`, `bicgstab`, `gmres`, ...
* Iterative methods for top eigen-pairs and singular pairs:  
    `eigs`, `svds`, ...
* Pre-conditioner:  
    `cholinc`, `luinc`, ...
    
* Get familiar with the **reverse communication interface (RCI)** for utilizing iterative solvers:
```matlab
x = gmres(A, b)
x = gmres(@Afun, b)
eigs(A)
eigs(@Afun)
```

### Julia

* `eigs` and `svds` in the [Arpack.jl](https://github.com/JuliaLinearAlgebra/Arpack.jl) package. [Numerical examples](http://hua-zhou.github.io/teaching/biostatm280-2019spring/slides/17-eigsvd/eigsvd.html#Lanczos/Arnoldi-iterative-method-for-top-eigen-pairs) later.

* [`IterativeSolvers.jl`](https://github.com/JuliaMath/IterativeSolvers.jl) package. [CG numerical examples](http://hua-zhou.github.io/teaching/biostatm280-2019spring/slides/15-iterative/iterative.html#Numerical-examples)

* See the [list](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1) of Julia packages for iterative methods.

#### Least squares example

We first generate a test problem. 

In [135]:
Random.seed!(257) # seed
n, p = 10000, 5000
X = sprandn(n, p, 0.005) # iid standard normals with sparsity 0.005
β = ones(p)
y = X * β + randn(n);

##### Dense QR decomposition (LAPACK)

In [136]:
β̂  =  Matrix(X) \ y
# least squares by sparse QR
@benchmark $(Matrix(X)) \ $y

BenchmarkTools.Trial: 1 sample with 1 evaluation per sample.
 Single result which took 10.496 s (0.00% GC) to evaluate,
 with a memory estimate of 383.24 MiB, over 32 allocations.

##### Sparse QR decomposition

In [137]:
β̂_qr = X \ y
@show norm(β̂ - β̂_qr)
# least squares by sparse QR
@benchmark $X \ $y

norm(β̂ - β̂_qr) = 6.222011513212679e-13


BenchmarkTools.Trial: 3 samples with 1 evaluation per sample.
 Range (min … max):  2.392 s …   2.490 s  ┊ GC (min … max): 0.10% … 3.37%
 Time  (median):     2.435 s              ┊ GC (median):    3.23%
 Time  (mean ± σ):   2.439 s ± 48.991 ms  ┊ GC (mean ± σ):  2.25% ± 1.85%

  █                       █                               █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.39 s         Histogram: frequency by time        2.49 s <

 Memory estimate: 2.26 GiB, allocs estimate: 204.

##### IterativeSolvers.jl

LSQR algorithm as implemented in the `IterativeSolvers.jl` package.

In [138]:
β̂_lsqr = IterativeSolvers.lsqr(X, y)
@show norm(β̂_qr - β̂_lsqr)
# least squares by lsqr
@benchmark IterativeSolvers.lsqr($X, $y)

norm(β̂_qr - β̂_lsqr) = 4.659591520381186e-6


BenchmarkTools.Trial: 280 samples with 1 evaluation per sample.
 Range (min … max):  17.027 ms …  22.097 ms  ┊ GC (min … max): 0.00% … 15.20%
 Time  (median):     17.685 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.878 ms ± 732.701 μs  ┊ GC (mean ± σ):  2.45% ±  3.33%

      ██▃                                                       
  ▃▅▇████▆▃▇▇▇▇█▇▅▄▄▅▆▃▅▅▄▄▅▅▂▂▁▄▄▂▂▃▁▁▃▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▃
  17 ms           Histogram: frequency by time         20.9 ms <

 Memory estimate: 7.10 MiB, allocs estimate: 557.

LSMR algorithm as implemented in the `IterativeSolvers.jl` package.

In [139]:
β̂_lsmr = IterativeSolvers.lsmr(X, y)
@show norm(β̂_qr - β̂_lsmr)
# least squares by lsmr
@benchmark IterativeSolvers.lsmr($X, $y)

norm(β̂_qr - β̂_lsmr) = 0.0006005238630945922


BenchmarkTools.Trial: 382 samples with 1 evaluation per sample.
 Range (min … max):  12.503 ms … 35.031 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.827 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.100 ms ±  1.339 ms  ┊ GC (mean ± σ):  1.09% ± 2.76%

   ▂▅█▇▃▁                                                      
  ▄██████▇▅▃▅▅▃▄▃▃▂▃▃▃▂▂▃▁▁▁▂▂▃▂▂▁▁▂▂▂▁▁▂▁▁▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂ ▃
  12.5 ms         Histogram: frequency by time        16.4 ms <

 Memory estimate: 2.88 MiB, allocs estimate: 262.

I don't know how to precondition the LSQR/LSMR algorithm in `IterativeSolvers.jl`.

##### Krylov.jl on CPU

LSQR algorithm as implemented in the `Krylov.jl` package.

In [140]:
# without preconditioner
β̂_lsqr = Krylov.lsqr(X, y)
@show norm(β̂_qr - β̂_lsqr[1])
# least squares by lsqr!
workspace = LsqrWorkspace(X, y)
@benchmark Krylov.lsqr!($workspace, $X, $y)

norm(β̂_qr - β̂_lsqr[1]) = 4.659591520281458e-6


BenchmarkTools.Trial: 306 samples with 1 evaluation per sample.
 Range (min … max):  16.062 ms … 41.923 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.266 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.372 ms ±  1.475 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▁▆▁▁ ▇ ▆ ▃▅█▃▂▃  ▁   ▁                                    
  ▅▆▇████▇███▇██████▇▇█▇▇▇█▃▅▃▃▃▃▁▃▁▄▄▁▃▁▁▁▁▁▁▃▁▁▁▁▁▁▃▁▃▁▃▁▁▃ ▄
  16.1 ms         Histogram: frequency by time          17 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [141]:
# with preconditioner
pl = Diagonal(vec(sum(abs2, X, dims=1)))
workspace = LsqrWorkspace(X, y)
β̂_lsqr = Krylov.lsqr!(workspace, X, y, N = pl, ldiv = true)
@show norm(β̂_qr - β̂_lsqr.x)
@benchmark Krylov.lsqr!($workspace, $X, $y, N = $pl, ldiv = true)

norm(β̂_qr - β̂_lsqr.x) = 4.041348037195206e-6


BenchmarkTools.Trial: 333 samples with 1 evaluation per sample.
 Range (min … max):  14.784 ms …  15.692 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     15.033 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.036 ms ± 154.392 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁▁ ▆▄▁  ▃ ▃▄ ▄ ▅█ ▃▆▃ ▆ ▂                                 
  ▆▆▄▆███████▇█▇██▇█▆████████▄█▇█▄▇▃▄▅▄▆▃▁▃▃▃▃▃▄▁▁▁▃▁▄▁▁▃▁▁▁▄▄ ▄
  14.8 ms         Histogram: frequency by time         15.6 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

LSMR algorithm as implemented in the `Krylov.jl` package.

In [142]:
# without preconditioner
workspace = LsmrWorkspace(X, y)
β̂_lsmr = Krylov.lsmr!(workspace, X, y)
@show β̂_lsmr
@show norm(β̂_qr - β̂_lsmr.x)
# least squares by lsmr
@benchmark Krylov.lsmr!($workspace, $X, $y)

β̂_lsmr = ┌──────────────────┬──────────────────┬────────────────────┐
│     LsmrWorkspace│      nrows: 10000│         ncols: 5000│
├──────────────────┼──────────────────┼────────────────────┤
│Precision: Float64│ Architecture: CPU│Storage: 351.721 KiB│
├──────────────────┼──────────────────┼────────────────────┤
│         Attribute│              Type│                Size│
├──────────────────┼──────────────────┼────────────────────┤
│                 m│             Int64│             8 bytes│
│                 n│             Int64│             8 bytes│
│                 x│   Vector{Float64}│          39.062 KiB│
│                Nv│   Vector{Float64}│          39.062 KiB│
│               Aᴴu│   Vector{Float64}│          39.062 KiB│
│                 h│   Vector{Float64}│          39.062 KiB│
│              hbar│   Vector{Float64}│          39.062 KiB│
│                Mu│   Vector{Float64}│          78.125 KiB│
│                Av│   Vector{Float64}│          78.125 KiB│
│           er

BenchmarkTools.Trial: 308 samples with 1 evaluation per sample.
 Range (min … max):  16.096 ms … 16.725 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.241 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.251 ms ± 94.629 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▇▅▁▃   ▁▂▂  ▇▄ ▄▆▂█  ▃▁▁█▂ ▂▂ ▃ ▄                          
  ▃▄█████▅▃███▆███▇████▇▇████████▅███▇▇▅▅▅▅▄▅▅▃▃▃▃▃▄▃▃▄▁▁▁▁▁▃ ▅
  16.1 ms         Histogram: frequency by time        16.5 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [143]:
# with preconditioner
pl = Diagonal(vec(sum(abs2, X, dims=1)))
workspace = LsmrWorkspace(X, y)
β̂_lsmr = Krylov.lsmr!(workspace, X, y, N = pl, ldiv = true)
@show β̂_lsmr
@show norm(β̂_qr - β̂_lsmr.x)

β̂_lsmr = ┌──────────────────┬──────────────────┬────────────────────┐
│     LsmrWorkspace│      nrows: 10000│         ncols: 5000│
├──────────────────┼──────────────────┼────────────────────┤
│Precision: Float64│ Architecture: CPU│Storage: 390.783 KiB│
├──────────────────┼──────────────────┼────────────────────┤
│         Attribute│              Type│                Size│
├──────────────────┼──────────────────┼────────────────────┤
│                 m│             Int64│             8 bytes│
│                 n│             Int64│             8 bytes│
│                 x│   Vector{Float64}│          39.062 KiB│
│                Nv│   Vector{Float64}│          39.062 KiB│
│               Aᴴu│   Vector{Float64}│          39.062 KiB│
│                 h│   Vector{Float64}│          39.062 KiB│
│              hbar│   Vector{Float64}│          39.062 KiB│
│                Mu│   Vector{Float64}│          78.125 KiB│
│                Av│   Vector{Float64}│          78.125 KiB│
│             

8.476970604284841e-6

In [144]:
# with preconditioner
pl = Diagonal(vec(sum(abs2, X, dims=1)))
workspace = LsmrWorkspace(X, y)
β̂_lsmr = Krylov.lsmr!(workspace, X, y, N = pl, ldiv = true)
@show β̂_lsmr
@show norm(β̂_qr - β̂_lsmr.x)
@benchmark Krylov.lsmr!($workspace, $X, $y, N = $pl, ldiv = true)

β̂_lsmr = ┌──────────────────┬──────────────────┬────────────────────┐
│     LsmrWorkspace│      nrows: 10000│         ncols: 5000│
├──────────────────┼──────────────────┼────────────────────┤
│Precision: Float64│ Architecture: CPU│Storage: 390.783 KiB│
├──────────────────┼──────────────────┼────────────────────┤
│         Attribute│              Type│                Size│
├──────────────────┼──────────────────┼────────────────────┤
│                 m│             Int64│             8 bytes│
│                 n│             Int64│             8 bytes│
│                 x│   Vector{Float64}│          39.062 KiB│
│                Nv│   Vector{Float64}│          39.062 KiB│
│               Aᴴu│   Vector{Float64}│          39.062 KiB│
│                 h│   Vector{Float64}│          39.062 KiB│
│              hbar│   Vector{Float64}│          39.062 KiB│
│                Mu│   Vector{Float64}│          78.125 KiB│
│                Av│   Vector{Float64}│          78.125 KiB│
│             

BenchmarkTools.Trial: 334 samples with 1 evaluation per sample.
 Range (min … max):  14.783 ms …  15.690 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.989 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.003 ms ± 133.150 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁     ▁▁ █▂▆▃█ ▅▁▆▅▂▄▆▃▁▂▁▂ ▁                               
  ▇▆█▆▆▄▅▆██▇█████▆████████████▆█▇▄▄▃▄▅▃▃▆▁▄▄▁▁▄▃▄▄▃▁▄▃▃▃▁▁▁▁▃ ▄
  14.8 ms         Histogram: frequency by time         15.4 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

##### Krylov.jl on GPU

Krylov.jl functions run on [GPUs](https://jso.dev/Krylov.jl/stable/). See my other notebook for details.

#### Use LinearMaps in iterative solvers

In many applications, it is advantageous to define linear maps indead of forming the actual (sparse) matrix. For a linear map, we need to specify how it acts on right- and left-multiplication on a vector. The [`LinearMaps.jl`](https://github.com/Jutho/LinearMaps.jl) package is exactly for this purpose and interfaces nicely with `IterativeSolvers.jl`, `Arnoldi.jl` and other iterative solver packages.

Applications:  
1. The matrix is not sparse but admits special structure, e.g., easy + low rank (PageRank), Kronecker proudcts, etc.  
2. Less memory usage. 
3. Linear algebra on a standardized (centered and scaled) sparse matrix.

Consider the differencing operator that takes differences between neighboring pixels

$$
\mathbf{D} = \begin{pmatrix}
-1 & 1 & & & \\
& -1 & 1 & & \\
& & \ddots & \\
& & & - 1 & 1 \\
1 & & & & -1
\end{pmatrix}.
$$

In [145]:
using LinearMaps, IterativeSolvers

# Overwrite y with A * x
# left difference assuming periodic boundary conditions
function leftdiff!(y::AbstractVector, x::AbstractVector) 
    N = length(x)
    length(y) == N || throw(DimensionMismatch())
    @inbounds for i in 1:N
        y[i] = x[i] - x[mod1(i - 1, N)]
    end
    return y
end

# Overwrite y with A' * x
# minus right difference
function mrightdiff!(y::AbstractVector, x::AbstractVector) 
    N = length(x)
    length(y) == N || throw(DimensionMismatch())
    @inbounds for i in 1:N
        y[i] = x[i] - x[mod1(i + 1, N)]
    end
    return y
end

# define linear map
D = LinearMap{Float64}(leftdiff!, mrightdiff!, 100; ismutating=true) 

100×100 FunctionMap{Float64,true}(leftdiff!, mrightdiff!; issymmetric=false, ishermitian=false, isposdef=false)

Linear maps can be used like a regular matrix.

In [146]:
@show size(D)
v = ones(size(D, 2)) # vector of all 1s
@show D * v
@show D' * v;

size(D) = (100, 100)
D * v = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
D' * v = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

If we form the corresponding dense matrix, it will look like

In [147]:
Matrix(D)

100×100 Matrix{Float64}:
  1.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0  -1.0
 -1.0   1.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   1.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0


If we form the corresponding sparse matrix, it will look like

In [148]:
using SparseArrays
sparse(D)

100×100 SparseMatrixCSC{Float64, Int64} with 200 stored entries:
⎡⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⎤
⎢⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⎦

In [149]:
using UnicodePlots
spy(sparse(D))

       ┌──────────────────────────────────────────┐    
     1 │⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁│ > 0
       │⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
       │⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀

Compute top singular values using iterative method (Arnoldi).

In [150]:
using Arpack
Arpack.svds(D, nsv = 3)

(SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}([0.10000000000003784 0.12610107456829575 -0.06401967660582586; -0.10000000000003065 -0.12987207165686732 0.05597539641260025; … ; 0.1000000000000521 0.11708293685000366 -0.07931951776561391; -0.10000000000004498 -0.12183241414850406 0.07181130038329539], [1.999999999999991, 1.999013120731463, 1.999013120731457], [0.10000000000003396 -0.10000000000002691 … 0.10000000000004843 -0.10000000000004114; 0.12804975793830697 -0.1315662173203481 … 0.11951664975119619 -0.12402794466205336; -0.06002715628725726 0.05186839557967478 … -0.07560271445022253 0.06794901723277247]), 3, 32, 526, [-0.04258949839207237, -0.14721604804533064, 0.011022034322333656, -0.05622695730309966, 0.18486947987539046, 0.06893624219396152, 0.058094777857784594, 0.029289396615192375, 0.07254854992344173, 0.00975416583922165  …  -0.024831913415879165, 0.13578084697251516, -0.02681766807280268, 0.08554983305702116, -0.04044885519724034, -0.04633598288333846, -0.048040

In [151]:
using LinearAlgebra
# check solution against the direct method for SVD
svdvals(Matrix(D))

100-element Vector{Float64}:
 2.0
 1.9990131207314632
 1.999013120731463
 1.996053456856543
 1.996053456856543
 1.99112392920616
 1.99112392920616
 1.9842294026289555
 1.9842294026289555
 1.9753766811902753
 1.9753766811902753
 1.9645745014573774
 1.9645745014573774
 ⋮
 0.37476262917144926
 0.31286893008046185
 0.31286893008046174
 0.25066646712860857
 0.25066646712860846
 0.18821662663702862
 0.1882166266370286
 0.12558103905862672
 0.12558103905862666
 0.06282151815625658
 0.06282151815625657
 1.7635897249928907e-16

Compute top eigenvalues of the Gram matrix `D'D` using iterative method (Arnoldi).

In [152]:
Arpack.eigs(D'D, nev = 3, which = :LM)

([3.9999999999999907, 3.996053456856537, 3.9960534568565333], [-0.09999999999998786 0.009399690541663409 0.14110863126584505; 0.09999999999998202 -0.0005208581322834926 -0.14142039706777043; … ; -0.10000000000000063 0.027011172214567336 0.13881785395111446; 0.09999999999999405 -0.018241426666785143 -0.14023997416271491], 3, 29, 477, [0.11143136146466825, 0.09379191938271311, 0.26066819357025, 0.04821186674086208, 0.00044236861085249316, -0.14800776405222132, 0.11112031736267719, -0.07442363053822573, 0.04267710288965029, -0.23889296427901438  …  -0.0065084972593795155, 0.05077322319654768, 0.1460426446406352, 0.026520846605774986, -0.11477751117771083, 0.060738880152664966, -0.16090083149646847, -0.13006045875353353, 0.12879338556316872, -0.05985490000934182])

## Further reading

* Chapter 5 of [Numerical Optimization](https://ucla.worldcat.org/title/numerical-optimization/oclc/209918411&referer=brief_results) by Jorge Nocedal and Stephen Wright (1999).

* Sections 11.3-11.5 of [Matrix Computations](https://ucla.worldcat.org/title/matrix-computations/oclc/824733531&referer=brief_results) by Gene Golub and Charles Van Loan (2013).